In [1]:
import tensorflow as tf
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
from PIL import Image

In [3]:
img = cv2.imread('/Users/francesco/repos/computer_vision_project/dataset/mix_dataset/categorized_images/benign/ISIC_0052212.jpg')
height, width = img.shape[:2]
print(f"Image size: {width}x{height}")

Image size: 224x224


In [7]:
img = cv2.imread('/Users/francesco/repos/computer_vision_project/dataset/ISIC2020/train/train_images/ISIC_9961808.jpg')
height, width = img.shape[:2]
print(f"Image size: {width}x{height}")

Image size: 6000x4000


In [ ]:
import tensorflow as tf
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np

# Setup paths
image_path = '/Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete'
yolo_dir = '/Users/francesco/repos/computer_vision_project/saved_models/yolo_trained/yolo11n_float16.tflite'
output_dir = Path("cropped_detections")
output_dir.mkdir(exist_ok=True)

model = YOLO(yolo_dir, task='detect')

conf_threshold = 0.2
paths = list(Path(image_path).glob("*.jpg"))
image_paths = [str(p) for p in paths]

print(f"\nFound {len(image_paths)} images to process")

for i, img_path in enumerate(image_paths):
    print(f"\nProcessing image {i+1}/{len(image_paths)}: {img_path}")
    
    try:
        results = model.predict(
            source=img_path,
            imgsz=608,
            conf=conf_threshold,
            save=False,
            device='cpu'
        )
        
        for result in results:
            boxes = result.boxes
            original_img = cv2.imread(img_path)
            
            if original_img is None:
                print(f"Error: Could not load image {img_path}")
                continue
                
            img_height, img_width = original_img.shape[:2]
            
            for box in boxes:
                confidence = box.conf.item()
                
                if confidence > conf_threshold:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    class_id = box.cls.item()
  
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(img_width, x2), min(img_height, y2)
                    
                    if x2 <= x1 or y2 <= y1:
                        continue
                    
                    cropped_img = original_img[y1:y2, x1:x2]
                    
                    # Save both original and processed versions
                    output_stem = f"img{i}_det_cls{int(class_id)}_conf{confidence:.2f}"
                    
                    # Save original crop
                    orig_output_path = output_dir / f"{output_stem}_original.jpg"
                    cv2.imwrite(str(orig_output_path), cropped_img)
                    
                    # # Save processed version (for verification)
                    # proc_output_path = output_dir / f"{output_stem}_processed.jpg"
                    # cv2.imwrite(str(proc_output_path), (cropped_img * 255).astype(np.uint8))
                    
                    print(f"\nDetection saved:")
                    print(f"Class: {int(class_id)}, Confidence: {confidence:.3f}")
                    print(f"Box coordinates: ({x1}, {y1}) to ({x2}, {y2})")
                    print(f"Original saved to: {orig_output_path}")
                    # print(f"Processed saved to: {proc_output_path}")
                    print(f"Processed shape: {cropped_img.shape}")
                    
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        continue
    
    print("-" * 50)

print("\nProcessing completed!")


Found 14 images to process

Processing image 1/14: /Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete/ISIC_0024345.jpg
Loading /Users/francesco/repos/computer_vision_project/saved_models/yolo_trained/yolo11n_float16.tflite for TensorFlow Lite inference...

image 1/1 /Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete/ISIC_0024345.jpg: 608x608 1 lesion, 102.3ms
Speed: 1.9ms preprocess, 102.3ms inference, 0.8ms postprocess per image at shape (1, 3, 608, 608)

Detection saved:
Class: 0, Confidence: 0.753
Box coordinates: (0, 12) to (594, 410)
Original saved to: cropped_detections/img0_det_cls0_conf0.75_original.jpg
Processed shape: (398, 594, 3)
--------------------------------------------------

Processing image 2/14: /Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete/ISIC_0024351.jpg

image 1/1 /Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete/ISIC_0024351.jpg: 608x608 1 lesion, 

In [13]:
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path

# Setup paths
cropped_dir = Path('/Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete')
custom_model_path = '/Users/francesco/repos/computer_vision_project/saved_models/skc_model.tflite'
results_dir = Path("custom_model_results")
results_dir.mkdir(exist_ok=True)

def preprocess_image(img_path, target_size=(224, 224)):
    img = tf.io.read_file(str(img_path))
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize_with_pad(img, target_size[0], target_size[1])
    return img.numpy()

def get_prediction_label(prediction_score):
    threshold = 0.5
    if prediction_score >= threshold:
        return "malignant"
    return "benign"

interpreter = tf.lite.Interpreter(model_path=custom_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input details: {input_details}")
print(f"Output details: {output_details}")
print(f"\nProcessing images from {cropped_dir}")

for img_path in cropped_dir.glob('*.jpg'):
    print(f"\nProcessing: {img_path.name}")
    
    try:
        # Preprocess image using same pipeline as training
        processed_img = preprocess_image(img_path)
        print(f"Processed image shape: {processed_img.shape}")
        
        # Add batch dimension
        input_data = np.expand_dims(processed_img, axis=0)
        
        # Run inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        # Get prediction
        prediction_score = output_data[0][0]
        prediction_label = get_prediction_label(prediction_score)
        
        # Save result with prediction label (using original image for saving)
        output_path = results_dir / f"{img_path.stem}_{prediction_label}_conf{prediction_score:.2f}.jpg"
        original_img = cv2.imread(str(img_path))
        cv2.imwrite(str(output_path), original_img)  # Save original image
        
        print(f"Prediction: {prediction_label}")
        print(f"Confidence score: {prediction_score:.4f}")
        print(f"Saved to: {output_path}")
        
    except Exception as e:
        print(f"Error processing {img_path.name}: {str(e)}")
        print(f"Error details: {type(e).__name__}")
        import traceback
        print(traceback.format_exc())
    
    print("-" * 50)


print("\nProcessing completed!")

Loading TFLite model...
Input details: [{'name': 'x', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'Identity', 'index': 298, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Processing images from /Users/francesco/repos/computer_vision_project/dataset/mix_dataset/tobedelete

Processing: ISIC_0024345.jpg
Processed image shape: (224, 224, 3)
Prediction: malignant
Confidence score: 0.7059
Saved to: custom_model_results/I

In [ ]:
#The code should inference the custom classification model with both yolo input and dataset input and return the accuracy of both the process

In [85]:
# cropped_dir = Path("/Users/francesco/repos/computer_vision_project/dataset/mix_dataset/categorized_images/malignant")
# custom_model_path = '/Users/francesco/repos/computer_vision_project/saved_models/skc_model.tflite' 

# results_dir = Path("custom_model_results")
# results_dir.mkdir(exist_ok=True)

# interpreter = tf.lite.Interpreter(model_path=custom_model_path)
# interpreter.allocate_tensors()

# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# for img_path in cropped_dir.glob('*.jpg'):
#     img = cv2.imread(str(img_path))
#     print(img.shape)
#     img_resiz = cv2.resize(img, (224,224))
#     print(img_resiz.shape)
#     if img is None:
#         print(f"Could not read image: {img_path}")
#         continue
    
#     input_data = img_resiz.astype(np.float32) 
#     input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension
    
#     interpreter.set_tensor(input_details[0]['index'], input_data)
    
#     # Run inference
#     interpreter.invoke()
    
#     output_data = interpreter.get_tensor(output_details[0]['index'])
    
#     prediction = np.argmax(output_data)
#     confidence = output_data[0][prediction]
    
#     output_path = results_dir / f"{img_path.stem}_pred{prediction}_conf{confidence:.2f}.jpg"
#     cv2.imwrite(str(output_path), img)
    
#     print(f"Image: {img_path.name}")
#     print(f"Prediction: {prediction}")
#     print(f"Confidence: {confidence:.2f}")
#     print("-" * 50)

(224, 224, 3)
(224, 224, 3)
Image: ISIC_0030858.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0030680.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_7927525.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0033389.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0032097.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0337631.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0032929.jpg
Prediction: 0
Confidence: 0.00
--------------------------------------------------
(224, 224, 3)
(224, 224, 3)
Image: ISIC_0026784.jpg
Prediction

KeyboardInterrupt: 